# Single Objective Optimization Auto-Tuning

In [1]:
import paradance as para

Load the offline dataset used for Bayesian automatic tuning.

`FILE_PATH` is the path where the training data is stored. `FILE_NAME` is the filename.

In [2]:
loader = para.CSVLoader(
    file_path=FILE_PATH,
    file_name=FILE_NAME,
)

Each individual optimization target is stored sequentially in the `targets` list. The fusion form can be very flexible, using `eval()` to convert the string into a valid fusion target expression. 

If there is only one target, we can use 

In [3]:
formula="targets[0]"

Select the features to be used for hyperparameter search

In [4]:
selected_columns=['pxtr_0', 'pxtr_1', 'pxtr_2', 'pxtr_3', 'pxtr_4']

We should construct a `Calculator`, to calculate the composite score based on the selected features. 

The calculation method for the composite score can be chosen as either `sum` or `product`, corresponding to the **additive** and **multiplicative** forms of the formulas.

In [5]:
cal = para.Calculator(
    df=loader.df, 
    selected_columns=selected_columns, 
    equation_type="product",
)

Determine the Bayesian search space: whether to search for the power term; whether to search for the first-order term of the features.

In [6]:
power = True
first_order = False

Next, construct the optimization target based on the above settings, and determine the direction of optimization, whether to maximize or minimize the fusion target expression.

The optimization process will be recorded in the `LOG_FILE` file.

In [7]:
ob = para.MultipleObjective(
    direction="minimize",
    weights_num=len(selected_columns),
    formula=formula,
    power=power,
    first_order=first_order,
    log_file=LOG_FILE,
)

Add an appropriate optimizer to the optimization target, such as the "portfolio" optimizer in this case, and set its coverage ratio to 0.95. 

`target_column` is the sub-target that this optimizer is used to optimize, such as "price".

In [8]:
ob.add_calculator(
    calculator=cal,
    flag="portfolio",
    hyperparameter=0.95,
    target_column=TARGET,
)

Begin optimization, and save the tuning log. `n_trials` represents the number of rounds for hyperparameter search.

In [9]:
ob.optimize(n_trials=600)